# Clasificación de textos

En esta clase, exploraremos datos de mensajes de texto y crearemos modelos para predecir si un mensaje es spam o no.

### Bibliotecas necesarias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix, hstack
from nltk.tokenize import RegexpTokenizer

### Lectura de datos

In [2]:
spam_data = pd.read_csv('spam.csv')
spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


### División de datos en conjuntos de entrenamiento y prueba

In [3]:
X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], spam_data['target'], random_state=0)

### Pregunta 1
¿Qué porcentaje de los documentos en `spam_data` son spam?

*Esta función debe devolver un valor flotante, el valor porcentual (es decir, $ ratio * 100 $).*

In [4]:
def respuesta_uno():
    spam_data['target'].mean()
    spam=spam_data[spam_data['target'] != 0]
    print(len(spam), len(spam_data))
    return spam_data['target'].mean()*100

In [5]:
respuesta_uno()

747 5572


13.406317300789663

### Pregunta 2

Ajustar y transformar los datos de entrenamiento `X_train` utilizando un `count_vectorizer` con parámetros predeterminados.

Luego, ajuste un modelo de clasificación Naive Bayes multinomial. Calcule medidas de exactitud, presición, recall y f1-score usando los datos de prueba transformados.

*Esta función debe devolver las cuatro medidas de evaluación como una lista con los 4 valores en el orden solicitado cada valor como flotante.*

In [6]:
def respuesta_dos():
    scores=[]
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    
    clf=MultinomialNB()
    clf.fit(X_train_vectorized, y_train)
    print("matriz Frecuencias:",X_train_vectorized.shape, "clases",len(y_train))
    
    X_test_vectorized=vect.transform(X_test)
    predictions = clf.predict(X_test_vectorized)
    print("matriz Frecuencias Test:",X_test_vectorized.shape, "prediction",len(predictions))
    
    scores.append(accuracy_score(y_test,predictions))
    scores.append(precision_score(y_test,predictions))
    scores.append(recall_score(y_test,predictions))
    scores.append(f1_score(y_test,predictions))
    
    spam=predictions[predictions!= 0]
    print('Predicciones Spam:',len(spam))
    print('Porcentaje Spam:',len(spam)/len(predictions)*100)
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(confusion_matrix(y_test, predictions))
    print(tn, fp, fn, tp)
    return scores

In [7]:
respuesta_dos()

matriz Frecuencias: (4179, 7354) clases 4179
matriz Frecuencias Test: (1393, 7354) prediction 1393
Predicciones Spam: 184
Porcentaje Spam: 13.208901651112706
[[1193    3]
 [  16  181]]
1193 3 16 181


[0.9863603732950467,
 0.9836956521739131,
 0.9187817258883249,
 0.9501312335958005]

### Pregunta 3

Ajustar y transformar los datos de entrenamiento `X_train` utilizando un `count_vectorizer` con parámetros predeterminados.

Luego, ajuste un modelo de clasificación Naive Bayes multinomial con suavizado (smoothing) `alpha = 0.1`. Encuentre el área bajo la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [8]:
def respuesta_tres():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    clf=MultinomialNB(alpha=0.1)
    clf.fit(X_train_vectorized, y_train)
    X_test_vectorized=vect.transform(X_test)
    predictions = clf.predict(X_test_vectorized)
    print(predictions)
    return roc_auc_score(y_test,predictions)

In [9]:
respuesta_tres()

[0 0 0 ... 0 0 1]


0.9720812182741116

### Pregunta 4

Ajustar y transformar los datos de entrenamiento `X_train` utilizando un `TfidfVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **3**.

Luego, ajuste un modelo de clasificador Naive Bayes multinomial con suavizado (smoothing) `alfa = 0.1` y calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [10]:
def respuesta_cuatro():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    clf=MultinomialNB(alpha=0.1)
    clf.fit(X_train_vectorized, y_train)
    X_test_vectorized=vect.transform(X_test)
    predictions = clf.predict(X_test_vectorized)
    return roc_auc_score(y_test,predictions)

In [11]:
respuesta_cuatro()

0.9416243654822335

### Pregunta 5

¿Cuál es la longitud promedio de los documentos (número de caracteres) para los documentos spam y no spam?

*Esta función debe devolver una tupla (longitud promedio no spam, longitud promedio de spam).*

In [12]:
def char_counter(text_list):
    counter = []
    for text in text_list:
        counter.append(len(text))
    return counter        

In [13]:
def respuesta_cinco(df):
    # Filtrado el dataframe #
    df_spam = df[(df.target == 1)] 
    df_no_spam = df[(df.target == 0)]
    spam_counter = char_counter(df_spam['text'])
    no_spam_counter = char_counter(df_no_spam['text'])
    return (sum(no_spam_counter)/len(df_no_spam.index), sum(spam_counter)/len(df_spam.index))

In [14]:
respuesta_cinco(spam_data)

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Pregunta 6

Ajustar y transformar los datos de entrenamiento `X_train` usando un `TfidfVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **5**.

Usando esta matriz de término de documento y una característica adicional, **la longitud del documento (número de caracteres)**, ajustar a un modelo de Clasificación de Vector de Soporte con regularización `C = 10000`. Luego calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [16]:
def respuesta_seis():
    # Obtención de longitud de los documentos de entrenamiento y prueba #
    document_len_train = np.array(char_counter(X_train))
    document_len_test = np.array(char_counter(X_test))
    
    # Obtención de valores TF-IDF para entrenamiento y prueba #
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    
    print(X_train_vectorized.shape)
    
    #Concatenación de características#
    X_train_concat = add_feature(X_train_vectorized, document_len_train)
    X_test_concat = add_feature(X_test_vectorized, document_len_test)
    
    print(X_train_concat.shape)
    
    #Entrenamiento y prueba del clasificador#
    clf = SVC(C = 10000)
    clf.fit(X_train_concat, y_train)
    predictions = clf.predict(X_test_concat)
    
    return roc_auc_score(y_test,predictions)

In [17]:
respuesta_seis()

(4179, 1468)
(4179, 1469)


0.9661689557407943

### Pregunta 7

¿Cuál es el número promedio de dígitos por documento para los documentos no spam y spam?

*Esta función debe devolver una tupla (promedio de # dígitos no es spam, promedio # dígitos spam).*

In [18]:
def get_feature_counter(text_list, tokenizer):
    counter = []
    for text in text_list:
        tokens = tokenizer.tokenize(text)
        counter.append(len(tokens))
    return counter

In [19]:
def respuesta_siete(df):
    # Filtrado el dataframe #
    df_spam = df[(df.target == 1)] 
    df_no_spam = df[(df.target == 0)]
    # conteo de digitos #
    digit_tokenizer = RegexpTokenizer(r'\d')
    spam_counter = get_feature_counter(df_spam['text'], digit_tokenizer)
    no_spam_counter = get_feature_counter(df_no_spam['text'], digit_tokenizer)
    return (sum(no_spam_counter)/len(df_no_spam.index), sum(spam_counter)/len(df_spam.index))

In [20]:
respuesta_siete(spam_data)

(0.2992746113989637, 15.759036144578314)

### Pregunta 8

Ajustar y transformar los datos de entrenamiento `X_train` usando un `TfidfVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **5** y usando **n-gramas de palabras n = 1 a n = 3** (unigramas, bigramas y trigramas).

Usando esta matriz de término-documento y las siguientes características adicionales:
* la longitud del documento (número de caracteres)
* **cantidad de dígitos por documento**

Ajustar un modelo de Regresión logística con regularización `C = 100`. Luego calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [21]:
def respuesta_ocho():
    # Obtención de longitud de los documentos de entrenamiento y prueba #
    document_len_train = np.array(char_counter(X_train))
    document_len_test = np.array(char_counter(X_test))
    
    # Obtención de digitos por documento de entrenamiento y prueba#
    digit_tokenizer = RegexpTokenizer(r'\d')
    document_digits_train = np.array(get_feature_counter(X_train, digit_tokenizer))
    document_digits_test = np.array(get_feature_counter(X_test, digit_tokenizer))
    
    # Obtención de valores TF-IDF para entrenamiento y prueba #
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    
    print(X_train_vectorized.shape)
    
    #Concatenación de características#
    X_train_concat_char = add_feature(X_train_vectorized, document_len_train)
    X_test_concat_char = add_feature(X_test_vectorized, document_len_test)
    
    X_train_concat = add_feature(X_train_concat_char, document_digits_train)
    X_test_concat = add_feature(X_test_concat_char, document_digits_test)
    
    print(X_train_concat.shape)
    
    #Entrenamiento y prueba del clasificador#
    clf = LogisticRegression(C = 100)
    clf.fit(X_train_concat, y_train)
    predictions = clf.predict(X_test_concat)
    
    return roc_auc_score(y_test,predictions)

In [22]:
respuesta_ocho()

(4179, 3383)
(4179, 3385)


/home/german/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9759031798040846

### Pregunta 9

¿Cuál es el número promedio de caracteres que no son palabras (cualquier cosa que no sea una letra, un dígito o un guión bajo) por documento para los documentos que no son spam y spam?

*Sugerencia: utilice las clases de caracteres `\ w` y` \ W`*

*Esta función debe devolver una tupla (promedio de # caracteres que no son palabras, no spam, promedio de # caracteres que no son palabras, spam).*

In [23]:
def pregunta_nueve(df):
    # Filtrado el dataframe #
    df_spam = df[(df.target == 1)] 
    df_no_spam = df[(df.target == 0)]
    # conteo de digitos #
    specialchar_tokenizer = RegexpTokenizer(r'\w')
    spam_counter = get_feature_counter(df_spam['text'], specialchar_tokenizer)
    no_spam_counter = get_feature_counter(df_no_spam['text'], specialchar_tokenizer)
    return (sum(no_spam_counter)/len(df_no_spam.index), sum(spam_counter)/len(df_spam.index)) 

In [24]:
pregunta_nueve(spam_data)

(53.73181347150259, 109.82463186077644)

### Pregunta 10

Ajustar y transformar los datos de entrenamiento `X_train` usando un `CountVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **5** y utilizando **caracteres n-grams desde n = 2 a n = 5.**

Para decirle a `CountVectorizer` que use caracteres n-grams, pase en `analyzer = 'char_wb'` que crea caracteres n-gramas solo del texto dentro de los límites de las palabras. Esto debería hacer que el modelo sea más robusto a los errores ortográficos.

Usando esta matriz término documento y las siguientes características adicionales:
* la longitud del documento (número de caracteres)
* cantidad de dígitos por documento
* **cantidad de caracteres que no son palabras (cualquier cosa que no sea una letra, dígito o guión bajo).**

Ajustar un modelo de Regresión logística con regularización `C = 100`. Luego calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

También **encuentre los 10 coeficientes más pequeños y los 10 más grandes del modelo** y devuélvalos junto con el AUC en una tupla.

La lista de los 10 coeficientes más pequeños debe ordenarse primero, la lista de los 10 coeficientes más grandes debe ordenarse primero.

Las tres características que se agregaron a la matriz de términos del documento deben tener los siguientes nombres si aparecen en la lista de coeficientes:
['longitud_doc', 'conteo_digito', 'caracteres_no_palabra']

*Esta función debe devolver una tupla `(AUC como flotante, lista de coeficientes más pequeños, lista de coeficientes más grande)`.*

In [25]:
def respuesta_10():
    # Obtención de longitud de los documentos de entrenamiento y prueba #
    document_len_train = np.array(char_counter(X_train))
    document_len_test = np.array(char_counter(X_test))
    
    # Obtención de digitos por documento de entrenamiento y prueba#
    digit_tokenizer = RegexpTokenizer(r'\d')
    document_digits_train = np.array(get_feature_counter(X_train, digit_tokenizer))
    document_digits_test = np.array(get_feature_counter(X_test, digit_tokenizer))
    
    # Obtención de caracteres especiales por documento de entrenamiento y prueba#
    specialchar_tokenizer = RegexpTokenizer(r'\w')
    document_specialchar_train = np.array(get_feature_counter(X_train, specialchar_tokenizer))
    document_specialchar_test = np.array(get_feature_counter(X_test, specialchar_tokenizer))
    
    # Obtención de valores Countvectorizer para entrenamiento y prueba #
    vect = TfidfVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    
    print(X_train_vectorized.shape)
    
    #Concatenación de características#
    X_train_concat_char = add_feature(X_train_vectorized, document_len_train)
    X_test_concat_char = add_feature(X_test_vectorized, document_len_test)
    
    X_train_concat_digit = add_feature(X_train_concat_char, document_digits_train)
    X_test_concat_digit = add_feature(X_test_concat_char, document_digits_test)
    
    X_train_concat = add_feature(X_train_concat_digit, document_specialchar_train)
    X_test_concat = add_feature(X_test_concat_digit, document_specialchar_test)
    
    print(X_train_concat.shape)
    
    #Entrenamiento y prueba del clasificador#
    clf = LogisticRegression(C = 100)
    clf.fit(X_train_concat, y_train)
    predictions = clf.predict(X_test_concat)
    
    added_features = ['longitud_doc', 'conteo_digito', 'caracteres_no_palabra']
    features_coefs = clf.coef_[0][-3:]
    
    features_dict = dict(zip(added_features, features_coefs))
    
    sorted_coef = sorted(clf.coef_[0])
    
    min_coef = sorted_coef[:10]
    max_coef = sorted_coef[-10:]
    
    for feature in added_features:
        for index in range(10):
            if min_coef[index] == features_dict.get(feature):
                min_coef[index] = feature
            elif max_coef[index] == features_dict.get(feature):
                max_coef[index] = feature
                
    print(features_dict)
    
    return (roc_auc_score(y_test,predictions), min_coef, max_coef)

In [26]:
respuesta_10()

(4179, 16314)
(4179, 16317)
{'longitud_doc': -0.1399951687319573, 'conteo_digito': 0.7458820324653969, 'caracteres_no_palabra': 0.1917936152108704}


/home/german/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(0.9805612617353955,
 [-3.945912526988008,
  -3.247943281242045,
  -3.22616162914201,
  -3.046142138087868,
  -2.897679896859826,
  -2.699966521619068,
  -2.6325556556724936,
  -2.5363792443320543,
  -2.4709558067322144,
  -2.3698342812270363],
 [3.695829977143551,
  3.8744466806017037,
  3.9138516154990723,
  3.943569390418778,
  3.9492591298235653,
  4.4219593305637055,
  4.46309756059203,
  4.512965699609701,
  4.732352683166704,
  5.408390926803798])